
A notebook for Knockoff simulation
==============
    
<font color=#000000 size=3 face="黑体">  This notebook can be used to control FDR by knockoff.It also contain serveral function which use other method(BHQ and permutation),which can give you the chance to compare them

The first part of the notebook provide basic function(for example,the function for choosing s)

In [57]:
import numpy as np
from math import log
import random as rd
import pandas as pd
import platform
from random import choices
import scipy.linalg as sl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from sklearn.linear_model import lasso_path
import time
from random import sample
from scipy.stats import norm
from cvxopt import matrix, solvers
%matplotlib inline
'''
first part of this notebook:basic function design for knockoff process
'''
def Data_Generate(n=3000,p=1000,A_set={3.5,-3.5},true_size=30,sigma=1,Cov_mat=101):
    if type(Cov_mat)==type(1):
        X=np.random.normal(0,1,(n,p))
    else:
        X=np.random.multivariate_normal(np.zeros((p)),Cov_mat,size=n)
    X_mean=np.mean(X,axis=0)
    X=X-X_mean
    X_var=np.power(np.var(X,axis=0),0.5)
    X=X/X_var
    X=X/(n**0.5)
    true_feature=pd.Series(range(p)).sample(n=true_size,replace=False).tolist()
    true_beta=choices(list(A_set),k=true_size)
    beta_all=np.zeros((p,1))
    for i,features in enumerate(true_feature):
        beta_all[features,0]=true_beta[i]
    Y=np.dot(X,beta_all)+np.random.normal(0,1,(n,1))
    return (X,true_feature,true_beta,Y,beta_all)
def s_SDP_construct(sigma,type_min=0.5):
    p=sigma.shape[0]
    lambdamin=min(np.linalg.eig(sigma)[0])
    c=matrix(np.array([-1.0]*p))
    mat_1=np.zeros((p,p*p))
    for i in range(p):
        mat_1[i,i*(p+1)]=1.0
    G=[matrix(-mat_1.T)]
    G+=[matrix(mat_1.T)]
    a=np.zeros((p,p))
    for i in range(p):
        a[i,i]=-lambdamin*type_min
    h=[matrix(a)]
    h+=[matrix(sigma)]
    sol = solvers.sdp(c, Gs=G, hs=h)
    s=list(sol['x'])
    print(max(s))
    print(min(s))
    sita=0.0000001
    while True:
        try:
            print(sl.cholesky(2*sigma-np.diag((1-sita)*np.array(s))))
            break
        except Exception:
            sita=sita*10
    return np.diag((1-3*sita)*np.array(s))
def Knockoff_Construct(X,s_choose='normal'):
    design_var=np.dot(X.T,X)
    p=X.shape[1]
    if s_choose=='normal':
        sita=0.0000001
        while True:
            try:
                s=min(2*min(np.linalg.eig(design_var)[0]),1)*np.identity(p)*(1-sita)
                print(sl.cholesky(2*design_var-s))
                break
            except Exception:
                sita=sita*10
    else:
        s=s_SDP_construct(design_var)
    uhat=sl.svd(X,full_matrices=True)[0][:,p:2*p]
    CTC=2*s-np.dot(s,np.dot(np.linalg.inv(design_var),s))
    C=sl.cholesky(CTC).T
    Xhat=np.dot(X,np.identity(p)-np.dot(np.linalg.inv(design_var),s))+np.dot(uhat,C)
    return Xhat
def Simplest_W(X,Xhat,Y):
    return np.where(np.abs(np.dot(X.T,Y))>=np.abs(np.dot(Xhat.T,Y)),np.abs(np.dot(X.T,Y)),-np.abs(np.dot(Xhat.T,Y)))
def Simplest_Z(X,Xhat,Y):
    return (np.abs(np.dot(X.T,Y)),np.abs(np.dot(Xhat.T,Y)))
def Lasso_Z(X,Xhat,Y):
    n=X.shape[0]
    p=X.shape[1]
    X_all=np.array([X.T,Xhat.T]).reshape((-1,n)).T
    begin=time.clock()
    alpha,coef,useless1=lasso_path(X_all,Y,tol=100)
    print(time.clock()-begin)
    #alpha,coef,useless1=lasso_path(X_all,Y,eps=1/(p*1000),n_alphas=p*1000)
    Z_True=[]
    Z_Knockoff=[]
    for i in range(p):
        try:
            Z_True.append(alpha[np.nonzero(coef[:,i,:].reshape(-1))[0][0]])
        except IndexError:
            Z_True.append(0)
        try:
            Z_Knockoff.append(alpha[np.nonzero(coef[:,i+p,:].reshape(-1))[0][0]])
        except IndexError:
            Z_Knockoff.append(0)
    return (np.array(Z_True),np.array(Z_Knockoff))
def Lasso_W(X,Xhat,Y):
    Z_True,Z_Knockoff=Lasso_Z(X,Xhat,Y)
    return np.where(Z_True>=Z_Knockoff,Z_True,-Z_Knockoff)
def Find_T(X,Xhat,Y,q,W_Function=Simplest_W):
    W=W_Function(X,Xhat,Y)
    sorted_W=np.sort(W[W>0])
    for item in sorted_W:
        qhat=len(W[W<=-item])/len(W[W>=item])
        if q>qhat:
            return item
    return 'inf'
def FDR_Cal(X,Xhat,Y,q,beta_all,W_Function=Simplest_W):
    T=Find_T(X,Xhat,Y,q,W_Function=W_Function)
    W=W_Function(X,Xhat,Y)
    out=np.array([W.reshape(-1).tolist(),beta_all.reshape(-1).tolist()])
    outcome=pd.DataFrame(out.T,columns=['W','beta'])
    choosen=outcome[outcome.W>=T]
    choosen_true=choosen[choosen.beta!=0]
    print('choosen:'+str(len(choosen)))
    print('choosen true:'+str(len(choosen_true)))
    return 1-len(choosen_true)/len(choosen)
def Plot_Out(X,Xhat,Y,q,beta_all,Z_Function=Simplest_Z,W_Function=Simplest_W):
    Z_True,Z_Knockoff=Z_Function(X,Xhat,Y)
    Z_X_NULL=[]
    Z_Y_NULL=[]
    Z_X_Not_NULL=[]
    Z_Y_Not_NULL=[]
    for i,item in enumerate(beta_all):
        if item==0:
            Z_X_NULL.append(Z_True[i])
            Z_Y_NULL.append(Z_Knockoff[i])
        else:
            Z_X_Not_NULL.append(Z_True[i])
            Z_Y_Not_NULL.append(Z_Knockoff[i])
    Z_max=np.max(np.abs(W_Function(X,Xhat,Y)))
    z= np.linspace(0, Z_max, 50)
    T=Find_T(X,Xhat,Y,q,W_Function=W_Function)
    T1=np.linspace(0, T, 50)
    T2=np.linspace(T, T, 50)
    plt.figure()
    plt.plot(z,z,'--',color='blue')
    plt.plot(T1,T2,color='black')
    plt.plot(T2,T1,color='black')
    p1=plt.scatter(Z_X_NULL, Z_Y_NULL, marker='s', c='black')
    p2=plt.scatter(Z_X_Not_NULL, Z_Y_Not_NULL, marker='s', c='red')
    plt.xlabel('Z True')
    plt.ylabel('Z Knockoff')
    plt.yticks([T],['T='+str(T)])
    plt.xticks([T],['T='+str(T)])
    plt.legend(handles=[p1,p2],labels=['NULL','Not NULL'],loc=0)
    plt.title('Outcome of Simulation')
    plt.show()
def covf1(i,j):
    return 0.9**abs(i-j)
def Cov_f(p,function=covf1):
    return np.fromfunction(function,(p,p))

    




The second part of this program provide serveral class of FDR controller.

In [58]:
class KnockOff:
    def __init__(self,W_type='simple',q=None,type_k='knockoff',type_s='normal'):
        self.W_type=W_type
        self.X='a'
        self.Y=None
        self.T=None
        self.W='a'
        self.Z_True='a'
        self.Z_Knockoff=None
        self.Xhat=None
        self.q=q
        self.type=type_k
        self.useless=np.array([1])
        self.type_s=type_s
    def fit(self,X,Y):
        n=X.shape[0]
        p=X.shape[1]
        if n>=(2*p):
            self.X=X
            self.Y=Y
        else:
            beta_ls=np.dot(np.linalg.inv(np.dot(X.T,X)),np.dot(X.T,Y))
            sigma_est=np.power(np.sum(np.power(Y-np.dot(X,beta_ls),2))/(n-p),0.5)
            self.X=np.vstack((X,np.zeros((2*p-n,p))))
            print(Y.shape)
            print(np.random.normal(0,sigma_est,(2*p-n)).shape)
            self.Y=np.vstack((Y,np.random.normal(0,sigma_est,(2*p-n,1))))
            print(self.Y.shape)
            
        self.Xhat=Knockoff_Construct(self.X,s_choose=self.type_s)
    def find_T(self):
        if self.T!=None:
            return 'We already have T'
        if type(self.X)!=type(self.useless):
            return 'We don"t even have X'
        if self.q==None:
            return 'We don"t even have q'
        if type(self.W)!=type(self.useless):
            if self.W_type=='simple':
                self.W=Simplest_W(self.X,self.Xhat,self.Y)
            elif self.W_type=='lasso':
                self.W=Lasso_W(self.X,self.Xhat,self.Y)
        #print(self.W)
        sorted_W=np.sort(self.W[self.W>0])
        if self.type=='knockoff':
            for item in sorted_W:
                qhat=len(self.W[self.W<=-item])/len(self.W[self.W>=item])
                if self.q>qhat:
                    return item
            return 'inf'
        else:
            for item in sorted_W:
                qhat=(len(self.W[self.W<=-item])+1)/len(self.W[self.W>=item])
                if self.q>qhat:
                    return item
            return 'inf'

    def find_W(self):
        if type(self.W)==type(self.useless):
            return 'We already have W'
        if type(self.X)!=type(self.useless):
            return 'We don"t even have X'
        if self.W_type=='simple':
            self.W=Simplest_W(self.X,self.Xhat,self.Y)
        elif self.W_type=='lasso':
            if type(self.Z_True)==type(self.useless):
                self.W=np.where(self.Z_True>=self.Z_Knockoff,self.Z_True,-self.Z_Knockoff)
                return 'ok'
            else:
                self.W=Lasso_W(self.X,self.Xhat,self.Y)
        return 'ok'
    def find_Z(self):
        if type(self.Z_True)==type(self.useless):
            return 'We already have Z'
        if type(self.X)!=type(self.useless):
            return 'We don"t even have X'
        if self.W_type=='simple':
            self.Z_True,self.Z_Knockoff=Simplest_Z(self.X,self.Xhat,self.Y)
        elif self.W_type=='lasso':
            self.Z_True,self.Z_Knockoff=Lasso_Z(self.X,self.Xhat,self.Y)
        return 'ok'
    def Power(self,beta):
        if type(self.X)!=type(self.useless):
            return 'We don"t even have X'
        if self.T==None:
            self.T=self.find_T()
        if self.T=='inf':
            print('ggT')
            return 0
        if type(self.W)!=type(self.useless):
            self.W=self.find_W()
        true=len(beta[beta!=0])
        out=np.array([self.W.reshape(-1).tolist(),beta.reshape(-1).tolist()])
        outcome=pd.DataFrame(out.T,columns=['W','beta'])
        choosen=outcome[outcome.W>=self.T]
        choosen_true=choosen[choosen.beta!=0]
        print('true:'+str(len(choosen)))
        print('choosen true:'+str(len(choosen_true)))
        return len(choosen_true)/true
    def FDR(self,beta):
        if type(self.X)!=type(self.useless):
            return 'We don"t even have X'
        if self.T==None:
            self.T=self.find_T()
        if self.T=='inf':
            print('ggT')
            return 0
        if type(self.W)!=type(self.useless):
            self.W=self.find_W()
        out=np.array([self.W.reshape(-1).tolist(),beta.reshape(-1).tolist()])
        outcome=pd.DataFrame(out.T,columns=['W','beta'])
        choosen=outcome[outcome.W>=self.T]
        choosen_true=choosen[choosen.beta!=0]
        print('choosen:'+str(len(choosen)))
        print('choosen true:'+str(len(choosen_true)))
        return 1-len(choosen_true)/len(choosen)
    def plot(self,beta,picname='knockoff'):
        if type(self.X)!=type(self.useless):
            return 'We don"t even have X'
        if type(self.Z_True)!=type(self.useless):
            self.Z_True,self.Z_Knockoff=self.find_Z()
        Z_X_NULL=[]
        Z_Y_NULL=[]
        Z_X_Not_NULL=[]
        Z_Y_Not_NULL=[]
        for i,item in enumerate(beta):
            if item==0:
                Z_X_NULL.append(self.Z_True[i])
                Z_Y_NULL.append(self.Z_Knockoff[i])
            else:
                Z_X_Not_NULL.append(self.Z_True[i])
                Z_Y_Not_NULL.append(self.Z_Knockoff[i])
        Z_max=np.max(np.array(np.max(np.abs(self.Z_True)),np.max(np.abs(self.Z_Knockoff))))
        z= np.linspace(0, Z_max, 50)
        if self.T==None:
            self.T=self.find_T()
        plt.figure()
        try:
            T1=np.linspace(0, self.T, 50)
            T2=np.linspace(self.T, self.T, 50)
            plt.plot(T1,T2,color='black')
            plt.plot(T2,T1,color='black')
            plt.xticks([self.T],['T='+str(self.T)])
        except TypeError:
            print(self.T)
        plt.plot(z,z,'--',color='blue')
        p1=plt.scatter(Z_X_NULL, Z_Y_NULL, marker='s', c='black')
        p2=plt.scatter(Z_X_Not_NULL, Z_Y_Not_NULL, marker='s', c='red')
        plt.xlabel('Z True')
        plt.ylabel('Z Knockoff')
        #plt.yticks([self.T],['T='+str(self.T)])
        plt.legend(handles=[p1,p2],labels=['NULL','Not NULL'],loc=0)
        plt.title(picname)
        plt.savefig(picname+'.jpg',quality=95)
        plt.show()
class Permutation_Filter:
    def __init__(self,X,Y,type_r='lasso',):
        self.X=X
        self.Y=Y
        self.type_r=type_r
        self.outcome=1
    def permutation(self):
        n=self.X.shape[0]
        p=self.X.shape[1]
        X_per=self.X[sample(range(n),n),:]
        X_all=np.array([self.X.T,X_per.T]).reshape((-1,n)).T
        if self.type_r=='lasso':
            alpha,coef,useless1=lasso_path(X_all,self.Y)
            lambda_True=[]
            lambda_Knockoff=[]
            for i in range(p):
                try:
                    lambda_True.append(alpha[np.nonzero(coef[:,i,:].reshape(-1))[0][0]])
                except IndexError:
                    lambda_True.append(0)
                try:
                    lambda_Knockoff.append(alpha[np.nonzero(coef[:,i+p,:].reshape(-1))[0][0]])
                except IndexError:
                    lambda_Knockoff.append(0)
            self.outcome=np.array(lambda_True+lambda_Knockoff)
    def plot(self,beta,picname='permutation'):
        p=len(beta)
        x_true=[]
        lambda_true=[]
        x_null=[]
        lambda_null=[]
        x_per=range(p,2*p)
        lambda_per=self.outcome[p:2*p]
        for i,item in enumerate(beta):
            if item==0:
                x_null.append(i)
                lambda_null.append(self.outcome[i])
            else:
                x_true.append(i)
                lambda_true.append(self.outcome[i])
        plt.figure()
        pt=plt.scatter(x_true, lambda_true, marker='s', c='red')
        pn=plt.scatter(x_null,lambda_null, marker='s', c='black')
        pp=plt.scatter(x_per,lambda_per,marker='s',c='blue')
        plt.xlabel('index')
        plt.ylabel('lambda')
        plt.legend(handles=[pt,pn,pp],labels=['true','null','permutated'],loc=0)
        plt.title(picname)
        plt.savefig(picname+'.jpg',quality=95)
        plt.show()

class BHQ:
    def __init__(self,X,Y,q,type_r='simple'):
        self.X=X
        self.Y=Y
        self.type=type_r
        self.choice=np.zeros(X.shape[0])
        self.q=q
    def choose(self):
        p=self.X.shape[1]
        n=self.X.shape[0]
        sigma=np.dot(self.X.T,self.X)
        beta_ls=np.dot(np.linalg.inv(sigma),np.dot(self.X.T,self.Y))
        sita_est=np.sum(np.power(self.Y-np.dot(self.X,beta_ls),2))/(self.X.shape[0]-self.X.shape[1])
        if self.type!='whiten':
            Z=(1/sita_est)*beta_ls.reshape(-1)/np.power(np.diag(np.linalg.inv(sigma)),0.5)
        else:
            lambda0=min(np.linalg.eig(sigma)[0])
            design_mat_0=np.power(sita_est,2)*((1/lambda0)*np.eye(p)-np.linalg.inv(sigma))
            Z_cor=np.random.multivariate_normal(np.zeros((p)),design_mat_0,size=1)
            Z=(Z_cor.reshape(-1)+beta_ls.reshape(-1))*np.power(lambda0,0.5)/(sita_est)
        if self.type=='log_factor':
            q=self.q/(0.577+log(p))
        else:
            q=self.q
        T_list=np.sort(np.abs(Z)).tolist()
        for i,item in enumerate(T_list):
            if(p*2*norm.cdf(-item)/(p-i))<q:
                T=item
                break
        for i in range(p):
            if Z[i]>=item:
                self.choice[i]=1
    def FDR(self,beta):
        p=self.X.shape[1]
        number_choose=0
        number_wrong=0
        for i in range(p):
            if self.choice[i]==1:
                number_choose+=1
                if beta[i]==0:
                    number_wrong+=1
        '''if number_choose!=0:
            print('choose:'+str(number_choose))
            print('wrong:'+str(number_wrong))
            print('FDP:'+str(number_wrong/number_choose))
        else:
            print('gg')'''
        if number_choose==0:
            return 0
        return number_wrong/number_choose
    def Power(self,beta):
        p=self.X.shape[1]
        true=len(beta[beta!=0])
        number_choose=0
        number_wrong=0
        for i in range(p):
            if self.choice[i]==1:
                number_choose+=1
                if beta[i]==0:
                    number_wrong+=1
        '''if number_choose!=0:
            print('choose:'+str(number_choose))
            print('wrong:'+str(number_wrong))
            print('FDP:'+str(number_wrong/number_choose))
        else:
            print('gg')'''
        return (number_choose-number_wrong)/true

The third part of this program define a comparision function which compare the outcome of all these method

In [59]:
def exp(n=3000,p=1000,true=30,sgn_size=3.5,time0=1,file=None,picname='standard_test',cov=None,
        W_type='lasso',type_k='knockoff',type_s='normal'):
    if cov is None:
        cov=np.eye(p)
    for i in range(time0):
        X,true_feature,true_beta,Y,beta_all=Data_Generate(n=n,p=p,A_set={sgn_size,-sgn_size},
                                                          true_size=true,Cov_mat=cov)
        ko1=KnockOff(W_type=W_type,q=0.2,type_k=type_k,type_s=type_s)
        begin=time.clock()
        ko1.fit(X,Y)
        print(time.clock()-begin)
        ko1.find_Z()
        ko1.find_W()
        ko1.find_T()
        ko1.plot(beta_all,picname=picname+str(i))
        if file!=None:
            file.write('FDR:'+str(ko1.FDR(beta_all))+'\n')
            file.write('Power:'+str(ko1.Power(beta_all))+'\n')
def Knockoff_exp(FDR_group,Power_group,i,j,X,Y,beta_all,type_k='knockoff',type_s='SDP'):
    ko1=KnockOff(W_type='lasso',q=0.2,type_k=type_k,type_s=type_s)
    begin=time.clock()
    ko1.fit(X,Y)
    print(time.clock()-begin)
    ko1.find_Z()
    ko1.find_W()
    ko1.find_T()
    if i==0:
        print(ko1.FDR(beta_all))
        FDR_group.append(ko1.FDR(beta_all))
        Power_group.append(ko1.Power(beta_all))
    else:
        print(ko1.FDR(beta_all))
        FDR_group[j]+=ko1.FDR(beta_all)
        Power_group[j]+=ko1.Power(beta_all)
    return FDR_group,Power_group
def BHq_exp(FDR_group,Power_group,i,j,X,Y,beta_all,type_r='log_factor'):
    bhq=BHQ(X,Y,0.2,type_r=type_r)
    bhq.choose()
    print(bhq.FDR(beta_all))
    FDR_group[j]+=bhq.FDR(beta_all)
    Power_group[j]+=bhq.Power(beta_all)
    return FDR_group,Power_group
def Knockoff_vs_BHq(cov=None,f_record=None):
    n=600
    p=200
    true=30
    sgn_size=3.5
    if f_record is None:
        f_record=open('gg.txt','a')
    time0=5
    FDR_group=[0,0,0,0,0,0,0]
    Power_group=[0,0,0,0,0,0,0]
    if cov is None:
        cov=np.eye(p)
    for i in range(time0):
        j=0
        X,true_feature,true_beta,Y,beta_all=Data_Generate(n=n,p=p,A_set={sgn_size,-sgn_size},
                                                          true_size=true,Cov_mat=cov)
        FDR_group,Power_group=Knockoff_exp(FDR_group,Power_group,i,j,X,Y,beta_all,type_k='knockoff',type_s='SDP')
        j+=1
        FDR_group,Power_group=Knockoff_exp(FDR_group,Power_group,i,j,X,Y,beta_all,type_k='knockoff+',type_s='SDP')
        j+=1
        FDR_group,Power_group=Knockoff_exp(FDR_group,Power_group,i,j,X,Y,beta_all,type_k='knockoff',type_s='normal')
        j+=1
        FDR_group,Power_group=Knockoff_exp(FDR_group,Power_group,i,j,X,Y,beta_all,type_k='knockoff+',type_s='normal')
        j+=1
        FDR_group,Power_group=BHq_exp(FDR_group,Power_group,i,j,X,Y,beta_all,type_r='log_factor')
        j+=1
        FDR_group,Power_group=BHq_exp(FDR_group,Power_group,i,j,X,Y,beta_all,type_r='whiten')
        j+=1
        FDR_group,Power_group=BHq_exp(FDR_group,Power_group,i,j,X,Y,beta_all,type_r='simple')
        j+=1
    FDR=[x/time0 for x in FDR_group]
    Power=[x/time0 for x in Power_group]
    exp_name=['knockoff(SDP)','knockoff+(SDP)','knockoff(equal value)','knockoff+(equal value)',
             'BHQ(log)','BHQ(whiten)','BHQ(normal)']
    for j,name in enumerate(exp_name):
        f_record.write(name+':FDR:'+str(FDR[j])+',Power:'+str(Power[j])+'\n')

        
def comparision_simulation(filename='knockoff.txt',filename2='compare5.txt'):
    #first of all,use the test setting in the paper to see the strength of knockoff
    f_record=open(filename,'a')
    f_record.write('standard setting:n:3000,p:1000,true feature:30,signal size:3.5,knockoff type:normal,cov type:I\n')
    exp(file=f_record,time0=0)
    #f_record.close()
    #Secondly,provide several experient.change:signal size,true data size,total size,n<2p,cov matrix,
    #choosing W,s_construct
    #signal size:
    f_record.write('small_signal_size(0.5)\n')
    exp(sgn_size=0.5,file=f_record,picname='small_signal_size',time0=0)
    
    #true data size
    f_record.write('more_true_feature(100)\n')
    exp(true=100,file=f_record,picname='more_true_feature',time0=0)
    f_record.write('more_true_feature(10)\n')
    exp(true=10,file=f_record,picname='less_true_feature',time0=0)
    #smaller total size
    f_record.write('smaller_total_size(300,100)\n')
    exp(n=300,p=100,true=10,sgn_size=1.5,file=f_record,picname='smaller_total_size',time0=0)
    #n<2p
    f_record.write('n_smaller_than_2p(1500)\n')
    exp(n=1500,file=f_record,picname='n_smaller_than_2p',time0=0)
    #cov mat
    f_record.write('special_cov:0.9\n')
    cov=Cov_f(1000)
    exp(cov=cov,file=f_record,picname='special_cov',time0=0)
    #choose W,cov sp
    f_record.write('cov_sp_W\n')
    cov=Cov_f(1000)
    exp(cov=cov,file=f_record,picname='cov_sp_W',W_type='simple',time0=0)
    #choose W
    f_record.write('sp_W\n')
    exp(file=f_record,picname='sp_W',W_type='simple',time0=0)
    #choose s
    f_record.write('SDP\n')
    exp(n=1500,p=500,true=15,file=f_record,picname='SDP',type_s='SDP',time0=0)
    #Thirdly,try permutation-based method
    n=3000
    p=1000
    true=30
    sgn_size=3.5
    f_record.write('setting:n:'+str(n)+',p:'+str(p)+',true feature:'+str(true)+
                   ',signal size:'+str(sgn_size)+',permutation\n')
    time0=0
    for i in range(time0):
        X,true_feature,true_beta,Y,beta_all=Data_Generate(n=n,p=p,A_set={sgn_size,-sgn_size},
                                                          true_size=true)
        per=Permutation_Filter(X,Y)
        begin=time.clock()
        per.permutation()
        print(time.clock()-begin)
        per.plot(beta_all,picname='permutation'+str(i))
    f_record.close()
    #Lastly,compare the FDR of knockoff and BHq in paper setting and else setting
    f_record=open(filename2,'a')
    #setting of the paper
    f_record.write('setting:n:600,p:200,true feature:30,signal size:3.5\n')
    Knockoff_vs_BHq(cov=None,f_record=f_record)
    #then new setting
    f_record.write('setting:n:600,p:200,true feature:30,signal size:3.5,sepcial cov:0.9\n')
    cov=Cov_f(200)
    Knockoff_vs_BHq(cov=cov,f_record=f_record)
    f_record.close()

In [65]:
comparision_simulation(filename='knockoff.txt')

     pcost       dcost       gap    pres   dres   k/t
 0: -1.0905e+02 -3.8190e+02  1e+03  2e+00  2e-17  1e+00
 1: -1.0770e+02 -1.7935e+02  2e+02  4e-01  2e-15  8e-01
 2: -9.3631e+01 -1.6572e+02  2e+02  4e-01  3e-15  8e-01
 3: -5.3053e+01 -7.5108e+01  6e+01  1e-01  3e-15  3e-01
 4: -4.6344e+01 -4.9619e+01  8e+00  2e-02  3e-15  3e-02
 5: -4.5647e+01 -4.7307e+01  4e+00  1e-02  2e-15  2e-02
 6: -4.4686e+01 -4.4991e+01  6e-01  2e-03  1e-15  3e-03
 7: -4.4534e+01 -4.4663e+01  3e-01  8e-04  1e-15  9e-04
 8: -4.4438e+01 -4.4460e+01  4e-02  1e-04  2e-15  2e-04
 9: -4.4427e+01 -4.4437e+01  2e-02  6e-05  2e-15  6e-05
10: -4.4419e+01 -4.4420e+01  2e-03  6e-06  2e-15  6e-06
11: -4.4419e+01 -4.4419e+01  3e-04  1e-06  2e-15  1e-06
12: -4.4418e+01 -4.4419e+01  2e-05  7e-08  1e-15  7e-08
Optimal solution found.
0.40148043407610934
0.09051530768865106
[[ 1.33783282e+00  1.52393634e-01  5.31867172e-02 ... -5.09999698e-04
  -1.87244559e-02  5.64493005e-02]
 [ 0.00000000e+00  1.29398592e+00 -5.17426929e-02

0.6203789999999572
choosen:27
choosen true:23
0.14814814814814814
choosen:27
choosen true:23
true:27
choosen true:23
[[ 1.28783621e+00  1.34159093e-01  8.01763865e-02 ... -3.17919828e-02
   3.67002716e-02 -7.97809882e-02]
 [ 0.00000000e+00  1.28082920e+00  6.77363661e-04 ... -2.75449157e-02
  -8.12063277e-02  4.07440160e-02]
 [ 0.00000000e+00  0.00000000e+00  1.28533785e+00 ...  1.35476487e-01
  -1.16108837e-01  3.56593618e-03]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  5.12048380e-01
  -2.48287091e-01  5.74282948e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   7.75100108e-01  4.03178957e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  2.19177552e-03]]
0.16938300000037998
0.6222760000018752
choosen:16
choosen true:15
0.0625
choosen:16
choosen true:15
true:16
choosen true:15
0.0
0.0
0.4
     pcost       dcost       gap    pres   dres   k/t
 0: -1.0873e+02 -3.8253e+02  1e+03  2e+00  2e-17  1e+00
 

0.6055359999991197
choosen:14
choosen true:14
0.0
choosen:14
choosen true:14
true:14
choosen true:14
[[ 1.28277398  0.00603709 -0.05468885 ... -0.06898438 -0.0251764
  -0.0199985 ]
 [ 0.          1.28275978  0.00552108 ... -0.09105007  0.01982709
   0.02914012]
 [ 0.          0.          1.28159578 ...  0.05170926 -0.08763244
   0.10579316]
 ...
 [ 0.          0.          0.         ...  0.64127621 -0.15336096
  -0.55245751]
 [ 0.          0.          0.         ...  0.          0.70622781
  -0.12112619]
 [ 0.          0.          0.         ...  0.          0.
   0.00559668]]
0.16497900000103982
0.6439250000003085
choosen:14
choosen true:14
0.0
choosen:14
choosen true:14
true:14
choosen true:14
[[ 1.28277398  0.00603709 -0.05468885 ... -0.06898438 -0.0251764
  -0.0199985 ]
 [ 0.          1.28275978  0.00552108 ... -0.09105007  0.01982709
   0.02914012]
 [ 0.          0.          1.28159578 ...  0.05170926 -0.08763244
   0.10579316]
 ...
 [ 0.          0.          0.         ...  0.641

13: -3.7777e+00 -3.7777e+00  6e-05  6e-08  2e-15  1e-07
14: -3.7777e+00 -3.7777e+00  7e-06  8e-09  2e-15  2e-08
15: -3.7777e+00 -3.7777e+00  1e-06  1e-09  8e-16  3e-09
Optimal solution found.
0.05699322623973819
0.007326015976229686
[[ 1.39833614  1.28003109  1.14549693 ... -0.02921952 -0.01004429
  -0.01047301]
 [ 0.          0.59514231  0.57827645 ...  0.02266432 -0.0202194
  -0.0278756 ]
 [ 0.          0.          0.5795742  ...  0.06897007  0.08224355
   0.05334355]
 ...
 [ 0.          0.          0.         ...  0.45220484  0.45569271
   0.40916733]
 [ 0.          0.          0.         ...  0.          0.42324724
   0.40866055]
 [ 0.          0.          0.         ...  0.          0.
   0.39383662]]
45.077556000000186
0.47394100000019534
choosen:22
choosen true:9
0.5909090909090908
choosen:22
choosen true:9
true:22
choosen true:9
[[ 1.40381478  1.27503554  1.14102643 ... -0.02910548 -0.01000509
  -0.01043214]
 [ 0.          0.58735025  0.60539555 ...  0.02246892 -0.02065817
  -0

 1: -3.1196e+01 -6.2178e+01  1e+02  7e-02  2e-15  1e-01
 2: -2.0075e+01 -4.4764e+01  9e+01  6e-02  2e-15  1e-01
 3: -7.3837e+00 -1.1331e+01  1e+01  9e-03  2e-15  2e-02
 4: -5.4439e+00 -7.4370e+00  6e+00  5e-03  4e-15  1e-02
 5: -3.9966e+00 -4.1978e+00  5e-01  5e-04  2e-15  8e-04
 6: -3.9006e+00 -3.9691e+00  1e-01  2e-04  2e-15  3e-04
 7: -3.8788e+00 -3.9219e+00  9e-02  1e-04  1e-15  2e-04
 8: -3.8483e+00 -3.8562e+00  2e-02  2e-05  1e-15  4e-05
 9: -3.8447e+00 -3.8486e+00  8e-03  9e-06  2e-15  2e-05
10: -3.8418e+00 -3.8423e+00  1e-03  1e-06  1e-15  3e-06
11: -3.8416e+00 -3.8418e+00  5e-04  5e-07  1e-15  1e-06
12: -3.8414e+00 -3.8414e+00  7e-05  8e-08  1e-15  2e-07
13: -3.8414e+00 -3.8414e+00  4e-05  4e-08  2e-15  9e-08
14: -3.8414e+00 -3.8414e+00  4e-06  4e-09  1e-15  1e-08
15: -3.8414e+00 -3.8414e+00  1e-06  1e-09  1e-15  2e-09
Optimal solution found.
0.055277660863755615
0.007707165843315701
[[ 1.39453302  1.3059602   1.19486404 ...  0.04765954  0.08378788
   0.08629817]
 [ 0.        

0.5875629999973171
choosen:21
choosen true:10
0.5238095238095238
choosen:21
choosen true:10
true:21
choosen true:10
     pcost       dcost       gap    pres   dres   k/t
 0: -1.0075e+02 -3.9851e+02  1e+03  7e-01  2e-17  1e+00
 1: -3.1293e+01 -6.1918e+01  1e+02  7e-02  3e-15  1e-01
 2: -1.9070e+01 -4.2984e+01  9e+01  6e-02  5e-15  1e-01
 3: -7.1202e+00 -1.0671e+01  1e+01  8e-03  2e-15  2e-02
 4: -5.4191e+00 -7.2568e+00  5e+00  4e-03  3e-15  1e-02
 5: -4.0902e+00 -4.3105e+00  5e-01  5e-04  3e-15  1e-03
 6: -3.9733e+00 -4.0426e+00  2e-01  2e-04  1e-15  4e-04
 7: -3.9466e+00 -3.9865e+00  9e-02  9e-05  3e-15  2e-04
 8: -3.9177e+00 -3.9250e+00  2e-02  2e-05  1e-15  4e-05
 9: -3.9154e+00 -3.9203e+00  1e-02  1e-05  1e-15  3e-05
10: -3.9117e+00 -3.9125e+00  1e-03  2e-06  2e-15  4e-06
11: -3.9114e+00 -3.9117e+00  6e-04  7e-07  2e-15  2e-06
12: -3.9111e+00 -3.9112e+00  9e-05  9e-08  1e-15  2e-07
13: -3.9111e+00 -3.9111e+00  3e-05  3e-08  2e-15  7e-08
14: -3.9111e+00 -3.9111e+00  3e-06  3e-09  1e-